<a href="https://colab.research.google.com/github/kotguru/Nvidia_CUDA/blob/main/Gurov_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2200.000
BogoMIPS:            4400.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [ ]:
!apt install lshw

In [ ]:
!lshw -c video

  *-display
       description: 3D controller
       product: NVIDIA Corporation
       vendor: NVIDIA Corporation
       physical id: 4
       bus info: pci@0000:00:04.0
       version: a1
       width: 64 bits
       clock: 33MHz
       capabilities: msix pm bus_master cap_list
       configuration: driver=nvidia latency=0
       resources: iomemory:30-2f iomemory:30-2f irq:11 memory:c0000000-c0ffffff memory:380000000-38fffffff memory:390000000-391ffffff


In [ ]:
%%cu
#include <stdlib.h> 
#include <stdio.h> 
#include <string.h> 
#include <math.h> 
#include <time.h>

#define CIRCLE_CONSTRAIN 10
#define RIGTH_CONSTRAINT 350
#define TOP_CONSTRAINT 350
#define BOTTOM_CONSTRAINT 300


int M, N;
int An;

double *A; 
double *x; 
int *rep;

double *Device_A; 
double *Device_x; 
int *Device_rep;

long double T = 50.0;  
long double c = 1; 
long double den = 1; 
long double alph = 1; 
long double dx = 0.3; 
long double dy = 0.3;

void write_to(int id, int k, int num);
void fill_matrix(int num);


__global__ void straight_gauss(int n, double *A1, double* x1) 
{
    double a;

    unsigned int k = blockIdx.x * blockDim.x + threadIdx.x;
    if(k < n)
    {
        for (int j = k + 1; j < n; j++) 
        { 
            a = - A1[j * n + k] / A1[k * n + k]; 
            x1[j] += x1[k] * a; 
            
            for (int i = k ; i < n; i++) 
                A1[j * n + i] += A1[k * n + i] * a; 
        }
    }
}

__global__ void reverse_gauss(int n, double* A1, double* x1)
{
    x1[n - 1] /= A1[(n - 1) * (n + 1)]; 

    for (int i = n - 2; i >= 0; i--) 
    { 
        for (int j = i + 1; j < n; j++) 
            x1[i] -= x1[j] * A1[i * n + j]; 
        x1[i] /= A1[i * n + i]; 
    } 
}

int main(int argc, char* argv[]) 
{
    int numTr = 4;
    int r, z, y; 
    int nur;

    M = 8/dx; //размерность матрицы
    N = 6/dy; 
    An = 3/dx; 
    rep = (int*) malloc(sizeof(double) * N * M); 

    nur = 0; 
    r = 3/dx-1;
 
    for (int j = 0; j < N; j++)
    {
        for (int i = 0; i < M; i++) 
        {
            z=i-(5/dx);
            y=j-(3/dy);
            
            if ( ((z*z+y*y) >= r*r) &&  (i >= M-r) && (j >= N-r))
            {
                rep[j * M + i] = -1;
            }
            else 
            { 
                rep[j * M + i] = nur; 
                nur++; 
            } 
        }
    }

    A = (double*) malloc(sizeof(double) * nur * nur); //создаем массивы для коэффициентов и искомых значений
    x = (double*) malloc(sizeof(double) * nur); 

    memset(x, 0, sizeof(double) * nur ); 
    memset(A, 0, sizeof(double) * nur * nur); 
 
    fill_matrix(nur); 

    cudaMalloc((void**)&Device_rep, sizeof(int)*N* M);
    cudaMalloc((void**)&Device_x, sizeof(double) * nur);
    cudaMalloc((void**)&Device_A,sizeof(double) * nur * nur);

    cudaMemcpy(Device_rep, rep, sizeof(int) * N * M, cudaMemcpyHostToDevice);
    cudaMemcpy(Device_x, x, sizeof(double) * nur, cudaMemcpyHostToDevice);
    cudaMemcpy(Device_A, A, sizeof(double) * nur * nur, cudaMemcpyHostToDevice);

    cudaError cudaStatus;
    float timerValueGPU;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
 
    dim3 N_Treads(numTr);
    dim3 N_Block(nur * nur/numTr);
 
    cudaEventRecord(start, 0);
 
	printf("Straight gauss\n");
    straight_gauss <<< N_Block, N_Treads >>> (nur, Device_A, Device_x); 
    cudaStatus = cudaGetLastError();
    if (cudaStatus != cudaSuccess) 
    {
        //std::cout << "Straight gauss error:" << cudaGetErrorString(cudaStatus) << std::endl;
        printf("Straight gauss error: %s\n", cudaGetErrorString(cudaStatus));
        return 0;
    }
    cudaDeviceSynchronize();
 
 //   cudaMemcpy(rep, Device_rep, sizeof(int) * N * M, cudaMemcpyDeviceToHost);
 //   cudaMemcpy(x, Device_x, sizeof(double) * nur, cudaMemcpyDeviceToHost);
 //   cudaMemcpy(A, Device_A, sizeof(double) * nur * nur, cudaMemcpyDeviceToHost);

    printf("Reverse gauss\n");
    reverse_gauss <<< N_Block, N_Treads >>> (nur, Device_A, Device_x); 
    cudaStatus = cudaGetLastError();
    if (cudaStatus != cudaSuccess) 
    {
        //std::cout << "Reverse gauss error:" << cudaGetErrorString(cudaStatus) << std::endl;
        printf("Reverse gauss error: %s\n", cudaGetErrorString(cudaStatus));
        return 0;
    }
    cudaDeviceSynchronize();

    cudaMemcpy(rep, Device_rep, sizeof(int) * N * M, cudaMemcpyDeviceToHost);
    cudaMemcpy(x, Device_x, sizeof(double) * nur, cudaMemcpyDeviceToHost);
    cudaMemcpy(A, Device_A, sizeof(double) * nur * nur, cudaMemcpyDeviceToHost);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&timerValueGPU, start, stop);

    printf("Calculations time:  %f \n", float(timerValueGPU/1000));
    cudaFree(Device_rep);
    cudaFree(Device_A); 
    cudaFree(Device_x);
}

void write_to(int id, int k, int num) 
{ 
    double a = alph / c / den; 
    A[k * num + rep[id + M]] = a / dy / dy; 
    A[k * num + rep[id - M]] = a / dy / dy; 
    A[k * num + rep[id + 1]] = a / dx / dx; 
    A[k * num + rep[id - 1]] = a / dx / dx; 
    A[k * num + rep[id]] = - (2 * a / dx / dx) - (2 * a / dy / dy); 
}

void fill_matrix(int num) 
{ 
    int z, y;
    int k = 0; 
    int id; 
    float l, m;
    int r=3/dx-1;
    double a = alph / c / den; 
    //Перебираем всю матрицу

    for (int j = 0; j < N; j++) 
    { 
        for (int i = 0; i < M; i++) 
        {
            id = j * M + i; //номер текущего элемента
            z = i-(5/dx);
            y = j -(3/dy);
         
            if (((z*z+y*y) >= r*r) &&  (i >= M-r) && (j >= N-r)) //за круглой границей
            { 
                continue; 
            } 
            else 
            if (((z*z+y*y) <= r*r ) && ( ((z+1)*(z+1)+y*y) > r*r ) && ( (z*z+(y+1)*(y+1)) < r*r ) && (i > M-r) && (j > N-r))
            { 
                l =  (sqrt(r*r-y*y)-z) /dx;
                A[k * num + rep[id + M]] = a / dy / dy; 
                A[k * num + rep[id - M]] = a / dy / dy; 
                A[k * num + rep[id - 1]] = 2* a / (l+1) / dx / dx; 
                A[k * num + rep[id]] = - (2 * a  / l /dx / dx) - (2 * a / dy / dy); 
                x[k] = (-2) * a * CIRCLE_CONSTRAIN / l / (l+1) / dx / dx ;
            } 
            else 
            if (((z*z+y*y) <= r*r ) && ( ((z+1)*(z+1)+y*y) < r*r ) && ( (z*z+(y+1)*(y+1)) > r*r ) && (i > M-r) && (j > N-r))
            { 
                m =  (sqrt(r*r-z*z)-y) /dy;
                A[k * num + rep[id + 1]] = a / dx / dx; 
                A[k * num + rep[id - 1]] = a / dx / dx;  
                A[k * num + rep[id - M]] = 2* a / (m+1) / dy / dy;  
                A[k * num + rep[id]] = - (2 * a  / dx / dx) - (2 * a / m /dy / dy); 
                x[k] = (-2) * a * CIRCLE_CONSTRAIN / m / (m+1) / dy / dy ;
            } 
            else 
            if (((z*z+y*y) <= r*r ) && ( ((z+1)*(z+1)+y*y) >= r*r ) && ( (z*z+(y+1)*(y+1)) >= r*r ) && (i > M-r) && (j > N-r))     
            { 
                l =  (sqrt(r*r-y*y)-z) /dx;
                m =  (sqrt(r*r-z*z)-y) /dy;
                A[k * num + rep[id - M]] = 2* a / (m+1) / dy / dy; 
                A[k * num + rep[id - 1]] = 2* a / (l+1) / dx / dx; 
                A[k * num + rep[id]] = - (2 * a  / l /dx / dx) - (2 * a / m /dy / dy); 
                x[k] = ((-2) * a * CIRCLE_CONSTRAIN / l / (l+1) / dx / dx) + ((-2) * a * CIRCLE_CONSTRAIN / m / (m+1) / dy / dy);
            } 
            else 
            if (j == 0 ) //нижняя граница
            {
                A[k * num + rep[id]] = 1;
                x[k] = BOTTOM_CONSTRAINT;
            } 
            else 
            if (j == N - 1) //верхняя граница
            { 
                A[k * num + rep[id]] = 1;
                x[k] = TOP_CONSTRAINT;
            } 
            else 
            if (i == 0) //левая граница
            {
                A[k * num + rep[id+1]] = 1;
                A[k * num + rep[id]] = -1 - dx;
                x[k] = 0;
            }
            else 
            if (i == M - 1) //правая граница
            {
                A[k * num + rep[id]] = 1;
                x[k] = RIGTH_CONSTRAINT;
            }
            else 
            if ( (i == 40) && ( j == 30))
            { 
                x[k] = 10000;
                write_to(id, k, num);
            }
            else 
            { 
                write_to(id, k, num);
            }
          
            k++; 
        }
    } 
}

Straight gauss
Reverse gauss
Calculations time:  3.177509 

